In [1]:
!pip install transformers
!pip install faiss-cpu
!pip install datasets
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 70.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.2 MB/s eta 0:00:00:00:01


In [24]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import faiss
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import streamlit as st
import re

In [4]:
data_path = '/kaggle/input/ny-rental-properties-pricing/NY Realstate Pricing.csv'
df = pd.read_csv(data_path)
df.head()
df_cleaned = df[['latitude', 'longitude', 'neighbourhood', 'room_type', 'price', 'days_occupied_in_2019', 'minimum_nights', 'number_of_reviews']]
df_cleaned.dropna(inplace=True)
df_cleaned.head()

/tmp/ipykernel_30/3238720158.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.dropna(inplace=True)


,latitude,longitude,neighbourhood,room_type,price,days_occupied_in_2019,minimum_nights,number_of_reviews
0,40.75362,-73.98377,Midtown,Entire home/apt,225,15,10,48
1,40.68514,-73.95976,Brooklyn,Entire home/apt,89,188,1,295
2,40.74767,-73.97500,Manhattan,Entire home/apt,200,362,3,78
3,40.68688,-73.95596,Bedford-Stuyvesant,Private room,60,0,29,49
4,40.76489,-73.98493,Manhattan,Private room,79,141,2,454


In [5]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1) 
    return embeddings


sample_texts = df_cleaned['neighbourhood'].head(10).tolist()
embeddings = get_embeddings(sample_texts)
embeddings_np = embeddings.numpy()

index = faiss.IndexFlatL2(embeddings_np.shape[1])  
index.add(embeddings_np)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [60]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
def generate_response(query, context):
    input_text = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = gpt_tokenizer.encode(input_text, return_tensors="pt")
    outputs = gpt_model.generate(inputs, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2)
    return gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)


def limit_response_length(response, max_length=1500):
    truncated_response = response[:max_length]
    last_punctuation_pos = max(truncated_response.rfind(punc) for punc in ['.', '!', '?'])
    if last_punctuation_pos != -1:
        truncated_response = truncated_response[:last_punctuation_pos + 1]
    else:
        truncated_response = truncated_response.rstrip().rsplit(' ', 1)[0]
    return truncated_response.strip()
    

def get_relevant_context(query, top_k=3):
    query_embedding = get_embeddings([query]).numpy()
    distances, indices = index.search(query_embedding, top_k)
    contexts = df_cleaned['neighbourhood'].iloc[indices[0]].tolist()
    return " ".join(contexts) 

def answer_query(query):
    context = get_relevant_context(query)
    answer = generate_response(query, context)
    answer = limit_response_length(answer)
    return answer

In [61]:
query = "What is the average rent in the Brooklyn neighborhood?"
response = answer_query(query)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Context: Lower East Side Manhattan Manhattan
Question: What is the average rent in the Brooklyn neighborhood?
Answer: The average price of a single-family home in Brooklyn is $1,890 per square foot.
The average monthly rent for a two-bedroom apartment in Manhattan is about $2,000 per month. The median monthly income in New York City is around $50,500. In Manhattan, the median income is between $40,400 and $45,800. For a three-person family, it is more than $60,200. A four- or five-year-old child in a family of four in Queens is worth about as much as a four year old in NYC. If you are a married couple, you can expect to pay about half of your monthly mortgage payments in one year. You can also expect a mortgage payment of about one-third of the monthly payments. This means that you will have to make about two times as many payments as you would if you were married. It is important to note that the mortgage is not a guarantee of future income. There is no guarantee that your home will b

In [62]:
st.title('NY Rental Properties RAG Model')

if 'history' not in st.session_state:
    st.session_state.history = []

def handle_query(query):
    answer = answer_query(query)
    st.session_state.history.append({"query": query, "answer": answer})

for entry in st.session_state.history:
    st.write(f"**Question**: {entry['query']}")
    st.write(f"**Answer**: {entry['answer']}")
    st.write("---")  

query = st.text_input('Ask a question about NY rental properties:')

if query:
    handle_query(query)
    st.experimental_rerun()  

2024-11-29 19:24:27.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 19:24:27.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [63]:
queries_and_responses = []
def test_query(query):
    response = answer_query(query) 
    queries_and_responses.append({"query": query, "answer": response})

queries = [
    "What is the average rent in Brooklyn?",
    "What are the most popular room types?",
    "What is the price range for a 2-Bedroom apartment?",
    "How many reviews are there on properties in Manhattan?"
]

for query in queries:
    test_query(query)

for entry in queries_and_responses:
    print(f"**Question**: {entry['query']}")
    print(f"**Answer**: {entry['answer']}")
    print("-" * 50) 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Question**: What is the average rent in Brooklyn?
**Answer**: Context: Lower East Side Manhattan Manhattan
Question: What is the average rent in Brooklyn?
Answer: $1,000
Source: Rent-A-Car
The average monthly rent for a Manhattan apartment is $2,500.
In New York City, the median rent is about $3,200. In Manhattan, it is around $4,400. The average price of a New Yorker apartment in Manhattan is just $6,800.
--------------------------------------------------
**Question**: What are the most popular room types?
**Answer**: Context: Park Slope Manhattan Manhattan
Question: What are the most popular room types?
Answer: The most common room type is the one that is most likely to be the first to get a lot of attention.
The most important thing to remember is that the room is not a place to sit or sit down. It is a space where you can sit and relax. The room should be comfortable and comfortable for you. If you are not comfortable, you will not be able to enjoy the experience. You will be dis